# DATA20001 Deep Learning - Group Project
## Text project

**Due Thursday, May 20, before 23:59.**

The task is to learn to assign the correct labels to news articles.  The corpus contains ~850K articles from Reuters.  The test set is about 10% of the articles. The data is unextracted in XML files.

We're only giving you the code for downloading the data, and how to save the final model. The rest you'll have to do yourselves.

Some comments and hints particular to the project:

- One document may belong to many classes in this problem, i.e., it's a multi-label classification problem. In fact there are documents that don't belong to any class, and you should also be able to handle these correctly. Pay careful attention to how you design the outputs of the network (e.g., what activation to use) and what loss function should be used.
- You may use word-embeddings to get better results. For example, you were already using a smaller version of the GloVE  embeddings in exercise 4. Do note that these embeddings take a lot of memory. 
- In the exercises we used e.g., `torchvision.datasets.MNIST` to handle the loading of the data in suitable batches. Here, you need to handle the dataloading yourself.  The easiest way is probably to create a custom `Dataset`. [See for example here for a tutorial](https://github.com/utkuozbulak/pytorch-custom-dataset-examples).

In [1]:
import pickle

import numpy as np

import data

## Get the data

In [2]:
# import sys
# import os
# import zipfile
# from os.path import join
# from os.path import abspath
# from os.path import split
#
# import numpy as np
# import torch
# import torchvision
# from torchvision.datasets.utils import download_url
#
# root_dir = os.getcwd()
# if root_dir not in sys.path:
#     sys.path.append(root_dir)
#
# train_path = 'train'
#
# data_folder_name = 'text-training-corpus'
# DATA_FOLDER_DIR = os.path.abspath(os.path.join(root_dir, data_folder_name))
#
# data_zip_name = 'reuters-training-corpus.zip'
# DATA_ZIP_DIR = os.path.abspath(os.path.join(DATA_FOLDER_DIR, data_zip_name))
#
# with zipfile.ZipFile(DATA_ZIP_DIR) as zip_f:
#     zip_f.extractall(train_path)

data.extract_data(extraction_dir="train", data_dir="data", data_zip_name="reuters-training-corpus.zip")

The above command downloads and extracts the data files into the `train` subdirectory.

The files can be found in `train/`, and are named as `19970405.zip`, etc. You will have to manage the content of these zips to get the data. There is a readme which has links to further descriptions on the data.

The class labels, or topics, can be found in the readme file called `train/codes.zip`.  The zip contains a file called "topic_codes.txt".  This file contains the special codes for the topics (about 130 of them), and the explanation - what each code means.

The XML document files contain the article's headline, the main body text, and the list of topic labels assigned to each article.  You will have to extract the topics of each article from the XML.  For example: 
&lt;code code="C18"&gt; refers to the topic "OWNERSHIP CHANGES" (like a corporate buyout).

You should pre-process the XML to extract the words from the article: the &lt;headline&gt; element and the &lt;text&gt;.  You should not need any other parts of the article.

## Your stuff goes here ...

In [3]:
try:
    with open("train/docs.pkl", "rb") as f:
        docs = pickle.load(f)
    labels = np.load("train/labels.npy")
except:
    docs, labels = data.get_docs_labels("train/REUTERS_CORPUS_2")
    with open("train/docs.pkl", "wb") as f:
        pickle.dump(docs, f)
    np.save("train/labels.npy", labels)

print(len(docs))
print(labels.shape)

print(docs[0])
print(labels[0])

1
(1, 126)
**** HIGHLIGHTS **** AMSTERDAM - The Netherlands hosts summit of European Union leaders (June 16-17). MADRID - NATO holds summit to set the course for enlargement (July 8 and 9). LUXEMBOURG - Luxembourg hosts summit of European Union leaders (December 12-13). APRIL BRUSSELS (MODIFIED ITEM) - Conference of Bosnian donor countries originally scheduled for April has been POSTPONED to an unspecified date before June. MONDAY, APRIL 7 NOORDWIJK, Netherlands (NEW ITEM) - EU foreign ministers hold conclave on the inter-governmental conference (second of two days). NOORDWIJK, Netherlands (EXPANDED ITEM) - EU-Rio Group meeting involving South American countries and Mexico, Panama, Trinidad, Tobago and Costa Rica (To April 8). Honduras, Guyana and chairmen of the Organisation of American States, the InterAmerican Development Bank, Latin American Parliament, the Institute for European-Latin American Relations, the European Investment Bank also attend. Discussions focus on political deve

## Save your model

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [4]:
# torch.save(model.state_dict(), 'model.pkl')

The model file should now be visible in the "Home" screen of the jupyter notebooks interface.  There you should be able to select it and press "download".

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You will be asked to return your predictions a separate test set.  These should be returned as a matrix with one row for each test article.  Each row contains a binary prediction for each label, 1 if it's present in the image, and 0 if not. The order of the labels is the order of the label (topic) codes.

An example row could like like this if your system predicts the presense of the second and fourth topic:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0 ...
    
If you have the matrix prepared in `y` you can use the following function to save it to a text file.

In [5]:
# np.savetxt('results.txt', y, fmt='%d')